In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 5
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()

250


In [3]:
sys.box = box
sys.atoms = satoms

In [4]:
sys.find_neighbors(method="voronoi")

In [7]:
sys.atom.neighbors.voronoi.vertex.numbers

[[6,
  1,
  21,
  16,
  22,
  13,
  14,
  6,
  1,
  9,
  8,
  3,
  15,
  21,
  4,
  1,
  14,
  6,
  9,
  6,
  2,
  11,
  12,
  10,
  17,
  7,
  6,
  2,
  8,
  9,
  6,
  5,
  11,
  4,
  2,
  7,
  3,
  8,
  6,
  3,
  7,
  17,
  0,
  18,
  15,
  6,
  4,
  19,
  23,
  20,
  10,
  12,
  6,
  4,
  5,
  6,
  14,
  13,
  19,
  4,
  4,
  12,
  11,
  5,
  4,
  10,
  20,
  0,
  17,
  4,
  13,
  22,
  23,
  19,
  4,
  15,
  18,
  16,
  21,
  6,
  16,
  18,
  0,
  20,
  23,
  22],
 [4,
  1,
  20,
  19,
  18,
  6,
  1,
  21,
  22,
  6,
  7,
  20,
  6,
  1,
  18,
  16,
  17,
  4,
  21,
  4,
  2,
  17,
  16,
  15,
  6,
  2,
  11,
  3,
  23,
  4,
  17,
  6,
  2,
  15,
  0,
  12,
  10,
  11,
  4,
  3,
  11,
  10,
  9,
  6,
  3,
  9,
  5,
  6,
  22,
  23,
  4,
  4,
  23,
  22,
  21,
  4,
  5,
  8,
  7,
  6,
  6,
  5,
  9,
  10,
  12,
  13,
  8,
  6,
  7,
  8,
  13,
  14,
  19,
  20,
  4,
  12,
  0,
  14,
  13,
  6,
  14,
  0,
  15,
  16,
  18,
  19],
 [6,
  1,
  4,
  7,
  15,
  11,
  14,
  6,
  1,
  21,


In [16]:
%%timeit
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

3.04 ms ± 80.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
a = sys.calculate_q(4, averaged=True)

In [6]:
sys.atom.steinhardt.average.q4_norm

[0.036369648372665306,
 0.03636964837266538,
 0.03636964837266529,
 0.036369648372665375,
 0.03636964837266533,
 0.036369648372665306,
 0.036369648372665334,
 0.036369648372665354,
 0.036369648372665334,
 0.036369648372665354,
 0.036369648372665334,
 0.03636964837266525,
 0.036369648372665334,
 0.036369648372665285,
 0.036369648372665306,
 0.0363696483726653,
 0.03636964837266533,
 0.0363696483726653,
 0.03636964837266533,
 0.03636964837266527,
 0.03636964837266533,
 0.036369648372665375,
 0.03636964837266527,
 0.036369648372665354,
 0.03636964837266537,
 0.03636964837266529,
 0.036369648372665334,
 0.03636964837266535,
 0.036369648372665354,
 0.03636964837266535,
 0.03636964837266531,
 0.03636964837266529,
 0.036369648372665306,
 0.03636964837266532,
 0.0363696483726653,
 0.03636964837266532,
 0.03636964837266529,
 0.03636964837266532,
 0.03636964837266531,
 0.03636964837266531,
 0.03636964837266531,
 0.03636964837266535,
 0.03636964837266527,
 0.036369648372665354,
 0.036369648372665

In [7]:
sys.find_solids()

2000

In [11]:
atoms, boxdims = pcs.make_crystal('diamond', repetitions = [4, 4, 4])
sys = pc.System()
sys.box = boxdims
sys.atoms = atoms

sys.find_neighbors(method = 'cutoff', cutoff=0)
sys.calculate_angularcriteria()

In [12]:
sys.atom.angular_parameters.diamond_angle

[1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,
 1.8488927466117464e-32,


Try a fully python numpy based algorithm for finding nearest neighbor

In [10]:
def get_distance(x, box, dim=3):
    box_2 = box/2
    xdum = []
    for i in range(dim):
        xd = np.meshgrid(x[i], x[i])[1] - np.meshgrid(x[i], x[i])[0]
        xdum.append(xd)
    xd = np.array(xdum)
    for i in range(dim):
        xd[i] = np.where(xd[i] > box_2[i], xd[i]-box[i], xd[i])
        xd[i] = np.where(xd[i] < -box_2[i], xd[i]+box[i], xd[i])
    return xd

In [12]:
x = sys.atoms['positions']
box = sys.boxdims

In [16]:
x = np.array(x)

In [20]:
y = x.T

In [28]:
%%timeit
xd = get_distance(y, np.array(box))

224 ms ± 857 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
xd.shape

(3, 2000, 2000)

In [23]:
np.where(xd[]

array([[[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635, -3.127 , ...,  4.6905,  3.127 ,  1.5635],
        [ 1.5635,  0.    , -1.5635, ...,  6.254 ,  4.6905,  3.127 ],
    